# Limitations and challenges in mobility data: noise for a single device

In [ ]:
from IPython.display import Image

# Set to True to render the kepler map or altair charts when running the notebook in local
# If False, screenshots will be shown instead
INTERACTIVE_OUTPUT = False

# If the platform allows to render gifs, set to True:
RENDERING_GIF = False

# Scaffolding to save the maps in html format:
SAVE_HTML_MAPS = False

## Noise classification

In [ ]:
from helpers_visualisation import get_map, get_two_timestamp_histograms
from noisyfier import Noisyfier, Sink
from simulate_data import create_ideal_L_shape_trajectory__sydney

df_noiseless_trajectory = create_ideal_L_shape_trajectory__sydney()

noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.df_pings[20:30]

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_0_trajectory_without_noise.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(
        config=kmap.config.copy(),
        file_name="maps/noise_0_trajectory_without_noise.html",
    )

## Spatial stationary radial noise

In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.spatial_stationary_spread(spread=5, noise_model="gaussian")  # try also "uniform" and "gumbel"
noisifyier.df_pings[20:30]

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_1_spatial_stationary_noise.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(config=kmap.config, file_name="maps/noise_1_spatial_stationary_noise.html")

## Temporal stationary radial noise

In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.temporal_stationary_spread(spread=40, noise_model="gumbel")  # try also "gaussian" and "uniform"
noisifyier.df_pings[20:30]

In [ ]:
if INTERACTIVE_OUTPUT:
    display(
        get_two_timestamp_histograms(
            df_pings1=df_noiseless_trajectory.copy(),
            title1="Ground truth",
            df_pings2=noisifyier.df_pings,
            title2="Temporal Radial Noise (Gaussian model)",
        )
    )
else:
    display(Image("images/noise_2_temporal_stationary_spatial_noise.png"))

## Spatial varying radial noise

In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.spatial_varying_spread(
    lower_bound_sec=0,
    upper_bound_sec=5 * 60,
    lower_bound_spread_m=1,
    upper_bound_spread_m=30,
    keywords=("dwell",),
    noise_model="gaussian",  # can be "uniform", "gaussian", "gumbel"
    variational_model="periodic",  # can be "linear" or "periodic"
)
noisifyier.spatial_varying_spread(
    lower_bound_sec=0,
    upper_bound_sec=180,
    lower_bound_spread_m=1,
    upper_bound_spread_m=8,
    keywords=(
        "moving",
        "entering",
    ),
    noise_model="uniform",
    variational_model="linear",
)
noisifyier.df_pings[20:30]

In [ ]:
if RENDERING_GIF:
    display(Image("images/noise_3_1_spatial_varying_radial_noise_slow.gif"))

if INTERACTIVE_OUTPUT:
    display(
        get_two_timestamp_histograms(
            df_pings1=df_noiseless_trajectory.copy(),
            title1="Ground truth",
            df_pings2=noisifyier.df_pings,
            title2="Temporal Radial Noise (Gaussian model)",
        )
    )
else:
    display(Image("images/noise_3_spatial_varying_radial_noise.png"))

## Temporal varying radial noise


In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.temporal_varying_spread(
    lower_bound_sec=0,
    upper_bound_sec=60,
    lower_bound_spread_sec=1,
    upper_bound_spread_sec=20,
    noise_model="gumbel",  # can be "uniform", "gaussian", "gumbel"
    variational_model="periodic",  # can be "linear" or "periodic"
)
noisifyier.df_pings[20:30]

In [ ]:
if INTERACTIVE_OUTPUT:
    display(
        get_two_timestamp_histograms(
            df_pings1=df_noiseless_trajectory,
            title1="Ground truth",
            df_pings2=noisifyier.df_pings,
            title2="Temporal Progressive Radial Noise (Gaussian model)",
        )
    )
else:
    display(Image("images/noise_4_time_varying_radial_noise.png"))

## Missing points


In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)

noisifyier.missing_points(
    frequency=0.6,
    reliability_model="uniform",  # available options are "uniform", "bathtub", "inverted_bathtub", "triangular_increasing", "triangular_decreasing"
    keywords=(),
)

noisifyier.missing_points(
    frequency=0.4,
    reliability_model="bathtub",
    keywords=("moving",),
)

noisifyier.missing_points(frequency=0.90, reliability_model="triangular_increasing", keywords=("dwell",))

noisifyier.spatial_stationary_spread(
    spread=10,
    noise_model="gaussian",
    keywords=("dwell",),
)

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_5_missing_points.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(config=kmap.config, file_name="maps/noise_5_missing_points.html")

## Erratic points


In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.seed = 108
noisifyier.erratic_points(
    frequency=0.03,
    reliability_model="uniform",
    keywords=("moving",),
    buffer_width_meters=50,
    total_sampling_for_erratic_points=1000,
)

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_6_erratic_points.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(config=kmap.config, file_name="maps/noise_6_erratic_points.html")

## Gridding artefact

In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)
# noisifyier.spatial_stationary_spread(spread=5, noise_model="gaussian")
noisifyier.gridding(
    precision=3,
    frequency=1,
    reliability_model="uniform",
    # keywords = ('moving towards shop 1', )
)

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_7_gridding.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(config=kmap.config, file_name="maps/noise_7_gridding.html")

## Sink locations


In [ ]:
sink_parramatta = Sink(151.148183, -33.868556, 0.03, "triangular_increasing")
sink_rugby_club = Sink(151.176491, -33.889456, 0.03, "triangular_increasing")

noisifyier = Noisyfier(df_noiseless_trajectory)
noisifyier.sink_locations(list_sinks=[sink_parramatta, sink_rugby_club], keywords=("moving",))

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_8_sink_locations.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(config=kmap.config, file_name="maps/noise_8_sink_locations.html")

## Combining noise models


In [ ]:
noisifyier = Noisyfier(df_noiseless_trajectory)

noisifyier.spatial_stationary_spread(
    spread=7,
    keywords=(
        "moving",
        "entering",
    ),
    noise_model="gaussian",
)


noisifyier.temporal_varying_spread(
    lower_bound_sec=0,
    upper_bound_sec=60,
    lower_bound_spread_sec=1,
    upper_bound_spread_sec=20,
    noise_model="gumbel",  # can be "uniform", "gaussian", "gumbel"
    variational_model="periodic",  # can be "linear" or "periodic"
)

noisifyier.spatial_varying_spread(
    lower_bound_sec=0,
    upper_bound_sec=5 * 60,
    lower_bound_spread_m=1,
    upper_bound_spread_m=30,
    keywords=("dwell",),
    noise_model="gaussian",
    variational_model="periodic",
)

noisifyier.spatial_varying_spread(
    lower_bound_sec=0,
    upper_bound_sec=180,
    lower_bound_spread_m=1,
    upper_bound_spread_m=8,
    keywords=(
        "moving",
        "entering",
    ),
    noise_model="uniform",
    variational_model="periodic",
)

noisifyier.missing_points(
    frequency=0.6,
    reliability_model="uniform",
    keywords=(),
)

noisifyier.missing_points(
    frequency=0.4,
    reliability_model="bathtub",
    keywords=("moving",),
)

noisifyier.missing_points(frequency=0.95, reliability_model="triangular_increasing", keywords=("dwell",))

noisifyier.gridding(
    precision=3,
    frequency=0.05,
    reliability_model="uniform",
)

noisifyier.erratic_points(
    frequency=0.03,
    reliability_model="uniform",
    keywords=("moving",),
    buffer_width_meters=50,
    total_sampling_for_erratic_points=1000,
)

sink_rugby_club = Sink(151.176491, -33.889456, 0.03, "triangular_increasing")

noisifyier.sink_locations(
    list_sinks=[
        sink_rugby_club,
    ],
)

noisifyier.df_pings.head(15)

In [ ]:
kmap = get_map(noisifyier)

if INTERACTIVE_OUTPUT:
    display(kmap)
else:
    display(Image("images/noise_9_combined.png"))

if SAVE_HTML_MAPS:
    kmap.save_to_html(config=kmap.config, file_name="maps/noise_9_combined.html")